#### Importing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Libraries imported.


#### Importing the basic cleaned data
Property prices of Mumbai City source:https://www.99acres.com/property-rates-and-price-trends-in-mumbai

In [2]:
pp_1=pd.read_csv("Data/mumbai_property_price.csv")
pp_1[['Locality Name','buy_rate_avg','Rent_1B_avg','Rent_2B_avg','Rent_3B_avg']].dropna().head(10)

,Locality Name,buy_rate_avg,Rent_1B_avg,Rent_2B_avg,Rent_3B_avg
0,4 Bunglows,19826.5,27625.0,45900.0,56767.5
1,Aarey Milk Colony,7543.5,18417.5,22277.0,26614.5
6,Airoli,10561.5,14259.0,22567.5,32742.0
10,Ambedkar Nagar,16235.0,25239.5,38675.0,56104.0
21,Andheri (East),16809.0,25478.5,39634.5,54910.0
22,Andheri (West),21058.5,25712.5,41536.0,66139.0
26,Asha Nagar,16171.5,18025.5,24593.5,32961.5
29,Azad Nagar,17871.5,24420.0,40350.0,58378.0
36,Bandra (West),37400.0,49608.0,79957.5,151512.5
38,Belapur,9860.0,11465.0,24952.0,43741.0


### Assigning Pincodes to each locality

In [24]:
url="https://nominatim.openstreetmap.org/search/Andheri%20Mumbai?format=json&addressdetails=1&limit=1&polygon_svg=1"
result=requests.get(url).json()
result[0]['lat']

'19.1196976'

In [25]:
url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format("4 Bungalows","Mumbai")
result=requests.get(url).json()
pp_1['Longitude']=None
pp_1.head()

,Locality Name,buy_rate,buy_rate_trim,buy_rate_lower,buy_rate_higher,buy_rate_avg,Buy Rates Q/Q,Rent_1B,Rent_1B_trim,Rent_1B_Lower,Rent_1B_Upper,Rent_1B_avg,Rent_1B_persqf,Rent_2B,Rent_2B_trim,Rent_2B_Lower,Rent_2B_Upper,Rent_2B_avg,Rent_2B_persqf,Rent_3B,Rent_3B_trim,Rent_3B_Lower,Rent_3B_Upper,Rent_3B_avg,Rent_3B_persqf,Longitude
0,4 Bunglows,"Rs.18,828 - 20,825/sq. ft.","18,828 - 20,825",18828.0,20825.0,19826.5,-1.48%,"Rs.24,225 - 31,025","24,225 - 31,025",24225.0,31025.0,27625.0,139%,"Rs.43,350 - 48,450","43,350 - 48,450",43350.0,48450.0,45900.0,232%,"Rs.53,780 - 59,755","53,780 - 59,755",53780.0,59755.0,56767.5,286%,None
1,Aarey Milk Colony,"Rs.7,225 - 7,862/sq. ft.","7,225 - 7,862",7225.0,7862.0,7543.5,4.71%,"Rs.16,371 - 20,464","16,371 - 20,464",16371.0,20464.0,18417.5,244%,"Rs.21,216 - 23,338","21,216 - 23,338",21216.0,23338.0,22277.0,295%,"Rs.23,899 - 29,330","23,899 - 29,330",23899.0,29330.0,26614.5,353%,None
2,Abhyudaya Nagar,"Rs.26,605 - 28,858/sq. ft.","26,605 - 28,858",26605.0,28858.0,27731.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rs.63,954 - 68,395","63,954 - 68,395",63954.0,68395.0,66174.5,239%,NaN,NaN,NaN,NaN,NaN,NaN,None
3,Adaigaon,"Rs.4,462 - 4,802/sq. ft.","4,462 - 4,802",4462.0,4802.0,4632.0,7.92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,Adharwadi,"Rs.4,675 - 5,398/sq. ft.","4,675 - 5,398",4675.0,5398.0,5036.5,-7.26%,"Rs.5,142 - 6,078","5,142 - 6,078",5142.0,6078.0,5610.0,111%,"Rs.7,268 - 8,882","7,268 - 8,882",7268.0,8882.0,8075.0,160%,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:

for index, row in pp_1.iterrows():

    locality=row['Locality Name']

#initialize your variable to None 


#loop until you get the coordinates

    #while(lat_lng_coords is None):
    url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format(locality,"Mumbai")
    result=requests.get(url).json()
    if result!=[]:
        pp_1.loc[index,'Latitude']=result[0]['lat']
        pp_1.loc[index,'Longitude']=result[0]['lon']

    else:
        pp_1.loc[index,'Latitude']=None
        pp_1.loc[index,'Longitude']=None

        
#Assign Latitude and longitude row['Latitude']= lat_lng_coords[0] row['Longitude'] = lat_lng_coords[1]


In [ ]:
pp_1.head()

In [ ]:
pp_1.to_csv("pp_latlong.csv")

In [ ]:
## importing from already saved csv file to save time and repeated API calls
# Dropping the locality for which the coordinates are not being able to be fetched

In [ ]:
city_data=pd.read_csv("Data/pp_latlong.csv")

In [ ]:
city_data[['Locality Name','Latitude','Longitude']].dropna().drop_duplicates().head(10)

In [ ]:
city_data.drop(['index'],axis=1,inplace=True)
city_data.tail()
city_data['Latitude'].dropna(inplace=True)
city_data['Longitude'].dropna(inplace=True)
print(city_data.shape)
city_data.head()

### Plotting data to check how much extent is covered by the extracted lat longs

In [ ]:
##setting the mean latitude and longitude for the area under consideration

In [ ]:
latitude =city_data.Latitude.mean()
longitude =city_data.Longitude.mean()
print(latitude,longitude)

In [ ]:
# Plotting coordinates to check to what extent the area is covered by sample space

In [ ]:

venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around center



# add coordinates to the map as blue circle markers
for lat, lng, label in zip(city_data.Latitude, city_data.Longitude, city_data['Locality Name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7
        ).add_to(venues_map)

# saving map to html
venues_map.save(outfile='plotcheck.html')

In [ ]:
# creating easy to read labels for each coordinates with Locality rates and average property price

In [ ]:
city_pp=city_data[['Locality Name','buy_rate_avg','Latitude','Longitude']]
city_pp['price_label']=city_pp['Locality Name']+" -Rs."+city_pp['buy_rate_avg'].astype(str)+"/sq.ft"
city_pp.dropna(inplace=True)
city_pp.head()

In [ ]:
## grouping the data by latitude and longitude if in case there is some repeatation in the data

In [ ]:
city_plot=city_pp[['Latitude','Longitude','buy_rate_avg']].groupby(['Latitude','Longitude'],as_index=False).mean()
city_plot.head()

In [ ]:
## Checking the distribution of average price per square feet on a histogram

x=sns.distplot(city_pp['buy_rate_avg'], label='rate per square feet', bins=50, norm_hist=True, axlabel='rate per sq.feet in Rs.')

### clustering by price

In [ ]:
##Generating Heatmap based on the price

In [ ]:
from folium.plugins import HeatMap

max_amount = float(city_plot['buy_rate_avg'].max())

hmap = folium.Map(location=[latitude, longitude], zoom_start=11,)

hm_wide = HeatMap( list(zip(city_plot.Latitude, city_plot.Longitude, city_plot['buy_rate_avg'])),
                   min_opacity=0.5,
                   max_val=45000,
                   radius=10, 
                   blur=10,
                   max_zoom=5, 
                 )

hmap.add_child(hm_wide)
for lat, lng, label in zip(city_pp.Latitude, city_pp.Longitude, city_pp['price_label']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=0.2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.01
        ).add_to(hmap)
hmap.save(outfile= "price_heat_map.html")

In [ ]:
## Binning the property prices into bins so that they can be mapped as a cluster on geo-map

In [ ]:
bins= np.linspace(min(city_pp["buy_rate_avg"]),max(city_pp["buy_rate_avg"]),10)
group_names=range(1,10)
city_pp['price_binned']=pd.cut(city_pp['buy_rate_avg'],bins,labels=group_names,include_lowest=True)

In [ ]:
city_pp.sort_values('price_binned',inplace=True)
city_pp.head()

In [ ]:
## Genrating the map with the clustered prices color coded.

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(10)
ys = [i + x + (i*x)**2 for i in range(10)]
colors_array = cm.Reds(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, tag, cluster in zip(city_pp['Latitude'], city_pp['Longitude'], city_pp['price_label'],city_pp['price_binned']):
    label = folium.Popup(str(tag), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        blur=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)

map_clusters.save(outfile="price_cluster_map.html")

# Using four square apis to get venue details

In [ ]:
CLIENT_ID="RFBWD0MXEMM3ESEHAPAJMAPGOA4GF5NBWA15BU1CYSRLLWSZ"
CLIENT_SECRET="M3LQZF1E0YG0EQJ0IUMDUFDQSGZN0RHNFXDH335O44CYPAIR"
VERSION='20180604'
radius=1000
LIMIT=100



url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()

In [ ]:
results

In [ ]:
items=results['response']['groups'][0]['items']
items

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
city_call=city_pp[['Locality Name','Latitude','Longitude']].drop_duplicates()
city_call.shape

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if results!=[]:
        # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Mumbai_venues = getNearbyVenues(names=city_call['Locality Name'],
                                   latitudes=city_call['Latitude'],
                                   longitudes=city_call['Longitude']
                                  )

In [ ]:
Mumbai_venues.head()

In [ ]:
Mumbai_venues.to_csv(r"Mumbai_venues.csv")

# Stored the venue details on git to do away with the need of calling the apis over and over again

In [23]:
city_venues=pd.read_csv("https://raw.githubusercontent.com/girishtere/Applied-Data-Science-Capstone/master/Data/Mumbai_venues.csv")
city_venues.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Aarey Milk Colony,19.156129,72.870722,The Westin Mumbai Garden City,19.172654,72.860518,Hotel
1,1,Aarey Milk Colony,19.156129,72.870722,PVR Cinemas,19.174016,72.860485,Multiplex
2,2,Aarey Milk Colony,19.156129,72.870722,Film City,19.160960,72.886177,Event Space
3,3,Aarey Milk Colony,19.156129,72.870722,Starbucks,19.174177,72.860350,Coffee Shop
4,4,Aarey Milk Colony,19.156129,72.870722,British Brewing Company (BBC),19.174186,72.860504,Brewery


In [ ]:
venue_count= city_venues[['Neighborhood','Venue']].groupby('Neighborhood', as_index=False).count()
venue_count.rename(columns={'Neighborhood':'Locality Name'},inplace=True)
venue_count

In [ ]:
## Checking the distribution of venues

x=sns.distplot(venue_count['Venue'], label='number of venues', bins=50, norm_hist=True, axlabel='Distribution')

In [ ]:
venue_plot=pd.merge(venue_count,city_pp,on='Locality Name', how='left')
venue_plot.drop(['price_label'],1,inplace=True)
venue_plot.drop_duplicates(inplace=True)

In [ ]:
from folium.plugins import HeatMap

max_amount = float(venue_plot['Venue'].max())

hmap = folium.Map(location=[latitude, longitude], zoom_start=11,)

hm_wide = HeatMap( list(zip(venue_plot.Latitude, venue_plot.Longitude, venue_plot['Venue'])),
                   min_opacity=0.5,
                   max_val=max_amount,
                   radius=7, 
                   blur=10,
                   max_zoom=5, 
                 )

hmap.add_child(hm_wide)
for lat, lng, label in zip(venue_plot.Latitude, venue_plot.Longitude, venue_plot['Locality Name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=0.2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.01
        ).add_to(hmap)
hmap.save(outfile= "venue_heat_map.html")